In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.rcParams['image.cmap'] = 'gist_earth'
np.random.seed(98765)

In [9]:
import tensorflow as tf
tf.test.is_gpu_available()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.gpu_device_name()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4647385322042564884
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15862523495
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17652903154034894233
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:0d:00.0, compute capability: 6.0"
]


'/device:GPU:0'

In [3]:
import sys
# Add the tf_unet folder path to the sys.path list
sys.path.append('/data3/floraxue/cs282a/SegmentNet/')


In [4]:
from tf_unet import image_gen
from tf_unet import unet
from tf_unet import util, image_util

In [5]:
#Data input
#put your data here
path_images = "/data3/floraxue/cs282a/data/dataset_1/trainval/B_phase/"
generator = image_util.ImageDataProvider(path_images+"*.tif")
#TODO: need to enhance the borders and rethink the matrix vs bkg 

Number of files used: 136900
Number of channels: 1
Number of classes: 2


In [6]:
net = unet.Unet(channels=1, n_class=2, layers=4, features_root=16)

2019-05-02 05:14:15,208 Layers 4, features 16, filter size 3x3, pool size: 2x2


In [7]:
trainer = unet.Trainer(net, batch_size=32, verification_batch_size = 8, 
                       optimizer="momentum", opt_kwargs=dict(momentum=0.2))

In [8]:
path = trainer.train(generator, "./unet_B", training_iters=4300, epochs=3, display_step=1000)

2019-05-02 05:15:00,424 Removing '/data3/floraxue/cs282a/SegmentNet/B_net/prediction'
2019-05-02 05:15:00,426 Removing '/data3/floraxue/cs282a/SegmentNet/B_net/unet_B'
2019-05-02 05:15:00,428 Allocating '/data3/floraxue/cs282a/SegmentNet/B_net/prediction'
2019-05-02 05:15:00,430 Allocating '/data3/floraxue/cs282a/SegmentNet/B_net/unet_B'
2019-05-02 05:15:02,775 Verification error= 48.4%, loss= 0.6941
2019-05-02 05:15:03,225 Start optimization
2019-05-02 05:15:07,388 Iter 0, Minibatch Loss= 0.6093, Training Accuracy= 0.8892, Minibatch error= 11.1%
2019-05-02 05:32:25,318 Iter 1000, Minibatch Loss= 0.2142, Training Accuracy= 0.9221, Minibatch error= 7.8%
2019-05-02 05:49:04,011 Iter 2000, Minibatch Loss= 0.2642, Training Accuracy= 0.8934, Minibatch error= 10.7%
2019-05-02 06:05:56,099 Iter 3000, Minibatch Loss= 0.2053, Training Accuracy= 0.9205, Minibatch error= 7.9%
2019-05-02 06:23:27,583 Iter 4000, Minibatch Loss= 0.1372, Training Accuracy= 0.9473, Minibatch error= 5.3%
2019-05-02 06:

In [ ]:
from skimage import io
x_test = io.imread(path_images+'0000.tif')
y_test = io.imread(path_images+'0000_mask.tif')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")

In [ ]:
x_test.shape

In [ ]:
np.max(generator(4)[1][0,:,:,1])

In [ ]:
#x_test, y_test = generator(1)
h,w = x_test.shape
x = x_test.reshape(1,h,w,1)
prediction = net.predict("./unet_trained/model.ckpt", x)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")
mask = prediction[0,...,1] > 0.1
ax[2].imshow(mask, aspect="auto")
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")

In [ ]:
fig.tight_layout()
fig.savefig("../docs/toy_problem.png")